In [281]:
import pandas as pd


## Model Training

In [282]:
df = pd.read_csv('./data/finalTrain.csv')
df.head()


,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [283]:
df=df.drop(labels=['ID','Delivery_person_ID','Order_Date','Time_Orderd','Time_Order_picked',"Restaurant_latitude",'Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude'],axis=1)
df.head()





,Delivery_person_Age,Delivery_person_Ratings,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,36.0,4.2,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,21.0,4.7,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,23.0,4.7,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,34.0,4.3,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,24.0,4.7,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [256]:
## Independent and dependent features
X = df.drop(labels=['Time_taken (min)'],axis=1)
Y = df[['Time_taken (min)']]


In [257]:
Y.head()

,Time_taken (min)
0,46
1,23
2,21
3,20
4,41


In [258]:
X.head()

,Delivery_person_Age,Delivery_person_Ratings,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,multiple_deliveries,Festival,City
0,36.0,4.2,Fog,Jam,2,Snack,3.0,No,Metropolitian
1,21.0,4.7,Stormy,High,1,Meal,1.0,No,Metropolitian
2,23.0,4.7,Sandstorms,Medium,1,Drinks,1.0,No,Metropolitian
3,34.0,4.3,Sandstorms,Low,0,Buffet,0.0,No,Metropolitian
4,24.0,4.7,Fog,Jam,1,Snack,1.0,No,Metropolitian


In [259]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns
categorical_cols


Index(['Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Festival', 'City'],
      dtype='object')

In [260]:
numerical_cols

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Vehicle_condition',
       'multiple_deliveries'],
      dtype='object')

In [261]:
# Define the custom ranking for each ordinal variable
#Delivery_person_ID_ranking=df['Delivery_person_ID'].unique().tolist()
#Order_Date_ranking=df['Order_Date'].unique().tolist()
#Time_Orderd_ranking=df['Time_Orderd'].unique().tolist()
#Time_Order_picked_ranking=df['Time_Order_picked'].unique().tolist()

#multiple_deliveries=[ 'nan',0.,1.,2.,3.]
weather_ranking = ['nan', 'Sunny', 'Cloudy', 'Windy', 'Sandstorms', 'Stormy', 'Fog']
Road_traffic_densitys_ranking=['nan','Low','Medium','High','Jam']
Type_of_orders_ranking=['Snack','Buffet', 'Meal', 'Drinks']
type_of_vehicle_ranking = ['electric_scooter', 'motorcycle', 'scooter', 'bicycle']
festival_ranking = ['nan', 'No','Yes' ]
city_ranking = ['nan','Semi-Urban','Urban','Metropolitian' ]






In [262]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [263]:

            ## Creating the Numerical Pipeline
num_pipeline=Pipeline(steps=[
                                         ('imputer',SimpleImputer(strategy='median')),
                                         ('scaler',StandardScaler())])

            # Creating the Categorigal Pipeline
cat_pipeline=Pipeline(steps=[
                                         ('imputer',SimpleImputer(strategy='most_frequent')),
                                         ('ordinalencoder',OrdinalEncoder(categories=[weather_ranking,Road_traffic_densitys_ranking,
                                                                                        Type_of_orders_ranking,
                                                                                        festival_ranking,city_ranking])),
                                        ('scaler',StandardScaler())
                                        ])
            
            # Creating the preprocessor
preprocessor=ColumnTransformer([
                                            ('num_pipeline',num_pipeline,numerical_cols),
                                            ('cat_pipeline',cat_pipeline,categorical_cols)
                                            ])

In [264]:

## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=42)



In [265]:
df.head()

,Delivery_person_Age,Delivery_person_Ratings,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,multiple_deliveries,Festival,City,Time_taken (min)
0,36.0,4.2,Fog,Jam,2,Snack,3.0,No,Metropolitian,46
1,21.0,4.7,Stormy,High,1,Meal,1.0,No,Metropolitian,23
2,23.0,4.7,Sandstorms,Medium,1,Drinks,1.0,No,Metropolitian,21
3,34.0,4.3,Sandstorms,Low,0,Buffet,0.0,No,Metropolitian,20
4,24.0,4.7,Fog,Jam,1,Snack,1.0,No,Metropolitian,41


In [266]:

X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())


In [267]:
X_train

,num_pipeline__Delivery_person_Age,num_pipeline__Delivery_person_Ratings,num_pipeline__Vehicle_condition,num_pipeline__multiple_deliveries,cat_pipeline__Weather_conditions,cat_pipeline__Road_traffic_density,cat_pipeline__Type_of_order,cat_pipeline__Festival,cat_pipeline__City
0,0.954495,-0.108398,-0.028178,0.441723,-0.325786,-0.293488,1.347459,-0.141231,0.537626
1,1.480327,0.194710,1.165832,0.441723,0.255372,0.507865,-0.440226,-0.141231,0.537626
2,1.129772,0.497818,1.165832,-1.324947,0.836530,-1.094840,1.347459,-0.141231,-1.788732
3,-0.447724,1.104034,1.165832,-1.324947,0.836530,-0.293488,1.347459,-0.141231,0.537626
4,1.480327,0.194710,-1.222187,0.441723,-0.906943,1.309217,-1.334068,-0.141231,0.537626
...,...,...,...,...,...,...,...,...,...
31903,-0.272447,0.800926,1.165832,0.441723,-1.488101,-1.094840,1.347459,-0.141231,0.537626
31904,-0.097169,0.800926,1.165832,0.441723,-1.488101,-1.094840,1.347459,-0.141231,0.537626
31905,-0.973556,-1.017722,-1.222187,-1.324947,-0.325786,-1.094840,0.453616,-0.141231,0.537626
31906,-0.447724,-0.714614,-1.222187,0.441723,0.836530,-0.293488,-0.440226,-0.141231,0.537626


In [268]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error


In [269]:
regression=LinearRegression()
regression.fit(X_train,y_train)


LinearRegression()

In [270]:
regression.coef_


array([[ 2.19696083, -2.29028076, -1.78414942,  2.03865082,  1.18738649,
         3.19962721, -0.05334967,  1.63104013,  0.77575306]])

In [271]:
regression.intercept_


array([26.28936317])

In [272]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square


In [273]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 6.692648629732893
MAE: 5.290833162313995
R2 score 49.85925265172121


Lasso
Model Training Performance
RMSE: 7.143477770676708
MAE: 5.7179387202108165
R2 score 42.876586882729086


Ridge
Model Training Performance
RMSE: 6.692649303231163
MAE: 5.290835314315831
R2 score 49.85924256013858


Elasticnet
Model Training Performance
RMSE: 7.212608056300183
MAE: 5.791608613355774
R2 score 41.76562501998167




In [92]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 6.690874071547673
MAE: 5.291344600170071
R2 score 49.88583880088603


Lasso
Model Training Performance
RMSE: 7.143477770676708
MAE: 5.7179387202108165
R2 score 42.876586882729086


Ridge
Model Training Performance
RMSE: 6.690874769704949
MAE: 5.291346850891531
R2 score 49.885828342590834


Elasticnet
Model Training Performance
RMSE: 7.2120457386006525
MAE: 5.791313619011099
R2 score 41.77470493754195




In [280]:
import pickle

# Open the PKL file for reading
with open(r'C:\Users\atulk\PycharmProjects\pythonProject2\M_L\Project_1_zomato_time_taken\artifacts\preprocessor1.pkl', 'rb') as f:
    # Load the object from the PKL file
    obj = pickle.load(f)

# Print the loaded object
print(obj)


ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['Delivery_person_Age',
                                  'Delivery_person_Ratings',
                                  'Vehicle_condition', 'multiple_deliveries']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(c...
                                                                              'Sandstorms',
                                                                              'Stormy',
      